In [12]:
import pandas as pd
import glob
import json
import numpy as np
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.PDBIO import PDBIO

import multiprocessing

In [13]:
'''Cada uno de esos archivos tiene un archivo .json, de estos archivos quiero que agarre
el valor de la llave 1, le calcule la media.
'''

def get_mean_from_json(path):
    with open(path) as f:
        data = json.load(f)
    return np.mean(data['1']['plddt'])



'''
ahora quiero una funcion que reciba un archivo en formato .pdb y le sustituya los valores de plddt por un valor que tambien recibe como parametro la funcion. para esto debes usar le modulo biopython.
'''

def change_plddt(path, value):
    parser = PDBParser()
    structure = parser.get_structure('name', path)
    model = structure[0]
    for chain in model:
        for residue in chain:
            for atom in residue:
                atom.bfactor = value
    io = PDBIO()
    io.set_structure(structure)
    io.save(path)




def paralelizar_el_proceso(lista_pdb_json):  

    pdbfile = lista_pdb_json[0]
    jsonfile = lista_pdb_json[1]

    #extract mean
    value = get_mean_from_json(jsonfile)
    value = round(value, 2)

    #change plddt in PDBfiles
    change_plddt(pdbfile, value)

    return pdbfile, value



In [14]:
lista_parametros = []
for pdbfile in glob.glob('../genome_data_sets/query_proteomes/pdb_files/wheeler_lab_data/pdb_json/*.pdb'):

    jsonfile = pdbfile[:-4] + '.json'

    l = [pdbfile, jsonfile]
    lista_parametros.append(l)


In [15]:
num_threads = 30


# Create a pool of processes.
pool = multiprocessing.Pool(num_threads)

# Map the extract_values function to each file name in the list.
results = pool.map(paralelizar_el_proceso, lista_parametros)

# Close the pool.
pool.close()
pool.join()

In [33]:
df = pd.DataFrame(results)
df[0] = df[0].str.split('/', expand=True)[6].str[:-4]
df.to_csv('../report/protein_structure_pLDDT_mean_wheeler_lab.tsv', sep='\t', index=False, header=False)

## muevo los archivos modificados a la carpeta de descargas de AFDB para que sean ingresadas.

In [40]:


import glob
import shutil


for file in glob.glob('../genome_data_sets/query_proteomes/pdb_files/wheeler_lab_data/pdb_json/*.pdb'):
    
    gene = file.split('/')[-1][:-4]

    name = f'AF-wheelerlab_{gene}-F1-model_v4.pdb'

    path = '../genome_data_sets/query_proteomes/pdb_files/prot_structure_download_from_AlphaFoldDB/'
    destination = path + name

    shutil.move(file, destination)